In [4]:
## 포스트 일괄 등록 - 최신순 100개

import requests
import json
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError




# 워드프레스 설정
WORDPRESS_URL = "https://prayer-church.co.kr/"
WORDPRESS_API_URL = f"{WORDPRESS_URL}/wp-json/wp/v2"
POSTS_PER_PAGE = 100  # 한 번에 가져올 포스트 수 (최대 100)

# 구글 인덱싱 API 설정
SERVICE_ACCOUNT_FILE = "/Users/popi/Desktop/wp/prompt/black-outlet-426702-n1-98f06232b595.json"  # 서비스 계정 인증 파일

def get_wordpress_posts(page=1):
    """워드프레스 API에서 게시물을 가져옵니다."""
    try:
        response = requests.get(
            f"{WORDPRESS_API_URL}/posts",
            params={
                "page": page,
                "per_page": POSTS_PER_PAGE,
                "status": "publish",
                "_fields": "id,link"  # 필요한 필드만 요청
            }
        )
        response.raise_for_status()
        
        # 전체 페이지 수 확인
        total_pages = int(response.headers.get("X-WP-TotalPages", 1))
        
        return response.json(), total_pages
    except requests.exceptions.RequestException as e:
        print(f"워드프레스 API 요청 오류: {e}")
        return [], 0

def get_all_wordpress_posts():
    """모든 워드프레스 포스트 URL을 수집합니다."""
    all_posts = []
    page = 1
    total_pages = 1
    
    while page <= total_pages:
        posts, total_pages = get_wordpress_posts(page)
        if not posts:
            break
        
        all_posts.extend(posts)
        print(f"페이지 {page}/{total_pages} 처리 완료, {len(posts)}개 포스트 수집")
        page += 1
        time.sleep(1)  # API 호출 제한 방지를 위한 딜레이
    
    return all_posts

def authenticate_indexing_api():
    """구글 Indexing API 인증"""
    try:
        credentials = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE,
            scopes=['https://www.googleapis.com/auth/indexing']
        )
        service = build('indexing', 'v3', credentials=credentials)
        return service
    except Exception as e:
        print(f"인덱싱 API 인증 오류: {e}")
        return None

def submit_url_to_index(service, url):
    """인덱싱 API에 URL 인덱싱 요청"""
    try:
        response = service.urlNotifications().publish(
            body={
                'url': url,
                'type': 'URL_UPDATED'  # 콘텐츠 업데이트 알림
            }
        ).execute()
        print(f"URL 제출 성공: {url}")
        return True
    except HttpError as e:
        print(f"URL 제출 실패 ({url}): {e}")
        return False

def main():
    # 1. 워드프레스에서 모든 포스트 가져오기
    print("워드프레스 포스트 수집 중...")
    posts = get_all_wordpress_posts()
    
    if not posts:
        print("포스트를 찾을 수 없습니다.")
        return
    
    print(f"총 {len(posts)}개의 포스트를 찾았습니다.")
    
    # 2. 인덱싱 API 인증
    print("구글 인덱싱 API 인증 중...")
    service = authenticate_indexing_api()
    
    if not service:
        print("인덱싱 API 인증에 실패했습니다.")
        return
    
    # 3. 각 URL을 인덱싱 API에 제출
    print("URL 인덱싱 제출 시작...")
    success_count = 0
    
    for post in posts:
        post_url = post.get('link')
        if post_url:
            if submit_url_to_index(service, post_url):
                success_count += 1
            time.sleep(1)  # API 호출 제한 방지를 위한 딜레이
    
    print(f"인덱싱 요청 완료: 총 {len(posts)}개 중 {success_count}개 성공")

if __name__ == "__main__":
    main()

워드프레스 포스트 수집 중...
페이지 1/1 처리 완료, 14개 포스트 수집
총 14개의 포스트를 찾았습니다.
구글 인덱싱 API 인증 중...
URL 인덱싱 제출 시작...
URL 제출 성공: https://prayer-church.co.kr/%ec%b2%ad%eb%85%84-%ec%a3%bc%ec%9d%bc-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8/
URL 제출 성공: https://prayer-church.co.kr/%ec%b4%9d%eb%8f%99%ec%9b%90-%ec%a3%bc%ec%9d%bc-%ea%b8%b0%eb%8f%84%eb%ac%b8/
URL 제출 성공: https://prayer-church.co.kr/%ec%b6%94%ec%88%98%ea%b0%90%ec%82%ac-%ec%a3%bc%ec%9d%bc-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8/
URL 제출 성공: https://prayer-church.co.kr/6%ec%9b%94-%ec%a3%bc%ec%9d%bc-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8/
URL 제출 성공: https://prayer-church.co.kr/5%ec%9b%94-%ec%a3%bc%ec%9d%bc-%eb%82%ae%ec%98%88%eb%b0%b0-%ea%b8%b0%eb%8f%84%eb%ac%b8/
URL 제출 성공: https://prayer-church.co.kr/5%ec%9b%94-%ec%a3%bc%ec%9d%bc-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8-10%ea%b0%9c/
URL 제출 성공: https://prayer-church.co.kr/%ec%a3%bc%ec%9d%bc-%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8-100%ea%b0%9c/
URL 제출 성

In [5]:
## 페이지 일괄 등록 - 최신순 100개

import requests
import json
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# 워드프레스 설정
WORDPRESS_URL = "https://prayer-church.co.kr"  # 실제 워드프레스 사이트 URL
WORDPRESS_API_URL = f"{WORDPRESS_URL}/wp-json/wp/v2"
PAGES_PER_REQUEST = 100  # 한 번에 가져올 페이지 수 (최대 100)

# 구글 인덱싱 API 설정
SERVICE_ACCOUNT_FILE = "/Users/popi/Desktop/wp/prompt/black-outlet-426702-n1-98f06232b595.json"  # 서비스 계정 인증 파일

def get_wordpress_pages(page_number=1):
    """워드프레스 API에서 페이지를 가져옵니다."""
    try:
        response = requests.get(
            f"{WORDPRESS_API_URL}/pages",  # 포스트 대신 페이지 엔드포인트 사용
            params={
                "page": page_number,
                "per_page": PAGES_PER_REQUEST,
                "status": "publish",
                "_fields": "id,link"  # 필요한 필드만 요청
            }
        )
        response.raise_for_status()
        
        # 전체 페이지 수 확인
        total_pages = int(response.headers.get("X-WP-TotalPages", 1))
        
        return response.json(), total_pages
    except requests.exceptions.RequestException as e:
        print(f"워드프레스 API 요청 오류: {e}")
        return [], 0

def get_all_wordpress_pages():
    """모든 워드프레스 페이지 URL을 수집합니다."""
    all_pages = []
    page_number = 1
    total_page_numbers = 1
    
    while page_number <= total_page_numbers:
        pages, total_page_numbers = get_wordpress_pages(page_number)
        if not pages:
            break
        
        all_pages.extend(pages)
        print(f"페이지 목록 {page_number}/{total_page_numbers} 처리 완료, {len(pages)}개 페이지 수집")
        page_number += 1
        time.sleep(1)  # API 호출 제한 방지를 위한 딜레이
    
    return all_pages

def authenticate_indexing_api():
    """구글 Indexing API 인증"""
    try:
        credentials = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE,
            scopes=['https://www.googleapis.com/auth/indexing']
        )
        service = build('indexing', 'v3', credentials=credentials)
        return service
    except Exception as e:
        print(f"인덱싱 API 인증 오류: {e}")
        return None

def submit_url_to_index(service, url):
    """인덱싱 API에 URL 인덱싱 요청"""
    try:
        response = service.urlNotifications().publish(
            body={
                'url': url,
                'type': 'URL_UPDATED'  # 콘텐츠 업데이트 알림
            }
        ).execute()
        print(f"URL 제출 성공: {url}")
        return True
    except HttpError as e:
        print(f"URL 제출 실패 ({url}): {e}")
        return False

def main():
    # 1. 워드프레스에서 모든 페이지 가져오기
    print("워드프레스 페이지 수집 중...")
    pages = get_all_wordpress_pages()
    
    if not pages:
        print("페이지를 찾을 수 없습니다.")
        return
    
    print(f"총 {len(pages)}개의 페이지를 찾았습니다.")
    
    # 2. 인덱싱 API 인증
    print("구글 인덱싱 API 인증 중...")
    service = authenticate_indexing_api()
    
    if not service:
        print("인덱싱 API 인증에 실패했습니다.")
        return
    
    # 3. 각 URL을 인덱싱 API에 제출
    print("URL 인덱싱 제출 시작...")
    success_count = 0
    
    for page in pages:
        page_url = page.get('link')
        if page_url:
            if submit_url_to_index(service, page_url):
                success_count += 1
            time.sleep(1)  # API 호출 제한 방지를 위한 딜레이
    
    print(f"인덱싱 요청 완료: 총 {len(pages)}개 중 {success_count}개 성공")

if __name__ == "__main__":
    main()

워드프레스 페이지 수집 중...
페이지 목록 1/1 처리 완료, 79개 페이지 수집
총 79개의 페이지를 찾았습니다.
구글 인덱싱 API 인증 중...
URL 인덱싱 제출 시작...
URL 제출 성공: https://prayer-church.co.kr/posts/
URL 제출 성공: https://prayer-church.co.kr/
URL 제출 성공: https://prayer-church.co.kr/prayers-easy/
URL 제출 성공: https://prayer-church.co.kr/sabbath-prayer/
URL 제출 성공: https://prayer-church.co.kr/prayers-short/
URL 제출 성공: https://prayer-church.co.kr/december-prayers-2024/
URL 제출 성공: https://prayer-church.co.kr/november-prayers-2024/
URL 제출 성공: https://prayer-church.co.kr/october-prayers-2024/
URL 제출 성공: https://prayer-church.co.kr/september-prayers-2024/
URL 제출 성공: https://prayer-church.co.kr/august-prayers-2024/
URL 제출 성공: https://prayer-church.co.kr/july-prayers-2024/
URL 제출 성공: https://prayer-church.co.kr/june-prayers-2024/
URL 제출 성공: https://prayer-church.co.kr/may-prayers-2024/
URL 제출 성공: https://prayer-church.co.kr/april-prayers-2024/
URL 제출 성공: https://prayer-church.co.kr/march-prayers-2024/
URL 제출 성공: https://prayer-church.co.kr/february-pra

In [7]:
#개별 페이지 등록 - 최신순 101-200개

import requests
import json
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# 워드프레스 설정
WORDPRESS_URL = "https://prayer-church.co.kr"  # 실제 워드프레스 사이트 URL
WORDPRESS_API_URL = f"{WORDPRESS_URL}/wp-json/wp/v2"
PAGES_PER_REQUEST = 100  # 한 번에 가져올 페이지 수 (최대 100)

# 구글 인덱싱 API 설정
SERVICE_ACCOUNT_FILE = "/Users/popi/Desktop/wp/prompt/black-outlet-426702-n1-98f06232b595.json"  # 서비스 계정 인증 파일

def get_wordpress_pages_range(start_index=1, end_index=100):
    """특정 범위의 워드프레스 페이지를 가져옵니다."""
    all_pages = []
    current_index = 0
    page_number = 1
    total_page_numbers = 1
    
    # 페이지 번호별로 데이터 요청
    while page_number <= total_page_numbers and current_index < end_index:
        # API 요청
        response = requests.get(
            f"{WORDPRESS_API_URL}/pages",
            params={
                "page": page_number,
                "per_page": 100,  # 최대 100
                "status": "publish",
                "_fields": "id,link",
                "orderby": "date",
                "order": "desc"
            }
        )
        
        if response.status_code != 200:
            print(f"API 요청 실패: {response.status_code}")
            break
            
        # 전체 페이지 수 확인
        total_page_numbers = int(response.headers.get("X-WP-TotalPages", 1))
        pages = response.json()
        
        # 현재 페이지의 항목들 처리
        for page in pages:
            current_index += 1
            
            # 시작 인덱스보다 작으면 건너뜀
            if current_index < start_index:
                continue
                
            # 종료 인덱스에 도달하면 중단
            if current_index > end_index:
                break
                
            all_pages.append(page)
        
        print(f"페이지 목록 {page_number}/{total_page_numbers} 처리 완료, 범위 {start_index}-{end_index} 중 {len(all_pages)}개 수집")
        
        # 다음 페이지로 이동
        page_number += 1
        time.sleep(1)  # API 호출 제한 방지를 위한 딜레이
    
    return all_pages

# 메인 함수 수정
def main():
    # 101-200번째 페이지 가져오기
    print("워드프레스 페이지 수집 중...")
    pages = get_wordpress_pages_range(101, 200)
    
    if not pages:
        print("페이지를 찾을 수 없습니다.")
        return
    
    print(f"총 {len(pages)}개의 페이지를 찾았습니다.")
    
    
    # 2. 인덱싱 API 인증
    print("구글 인덱싱 API 인증 중...")
    service = authenticate_indexing_api()
    
    if not service:
        print("인덱싱 API 인증에 실패했습니다.")
        return
    
    # 3. 각 URL을 인덱싱 API에 제출
    print("URL 인덱싱 제출 시작...")
    success_count = 0
    
    for page in pages:
        page_url = page.get('link')
        if page_url:
            if submit_url_to_index(service, page_url):
                success_count += 1
            time.sleep(1)  # API 호출 제한 방지를 위한 딜레이
    
    print(f"인덱싱 요청 완료: 총 {len(pages)}개 중 {success_count}개 성공")

if __name__ == "__main__":
    main()

워드프레스 페이지 수집 중...
페이지 목록 1/1 처리 완료, 범위 101-200 중 0개 수집
페이지를 찾을 수 없습니다.
